In [1]:
import pandas as pd 
import sqlite3 
import numpy as np 
import datetime
import sys 
import os 
from ast import literal_eval
%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('/home/malcolm/WorkTransit/scripts/WorkTransitLib/')

In [3]:
import RawDirections as rd

In [4]:
con = sqlite3.connect('/home/malcolm/WorkTransit/data/transit.db')
cursor = con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name')
tables = cursor.fetchall()
tables

[('Raw_Responses',)]

In [5]:
df = pd.read_sql('select * from raw_responses', con)
df

,origin,destination,mode,departure_time,Raw Response,Date,Time
0,"120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-09-03 09:29:53.537614,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-09-03,09:29
1,"120 Benchley Pl, Bronx NY",809 Washington Street,transit,2021-09-03 09:29:53.537614,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-03,09:29
2,"120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-09-03 11:09:25.191728,"[{'bounds': {'northeast': {'lat': 40.9347924, ...",2021-09-03,11:09
3,"120 Benchley Pl, Bronx NY",809 Washington Street,transit,2021-09-03 11:09:25.191728,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-03,11:09
4,"120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-09-03 11:11:19.014920,"[{'bounds': {'northeast': {'lat': 40.9347924, ...",2021-09-03,11:11
...,...,...,...,...,...,...,...
143,"Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-09-06 19:00:09.699638,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-06,19:00
144,"120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-09-06 19:17:29.822629,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-09-06,19:17
145,"Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-09-06 19:17:29.822629,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-09-06,19:17
146,"120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-09-06 19:30:13.377936,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-09-06,19:30


In [6]:
def parse_time(x):
    return(str(datetime.datetime.utcfromtimestamp(x)))
def parse_duration(x):
    return(np.round(x/3600.0, 2))
def parse_distance(x):
    return(np.round(x/1609.34, 2))

In [7]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

## Driving

In [84]:
temp = df[(df['destination'] == rd.work) &
   (df['mode'] == 'driving')].iloc[1]
driving_resp = literal_eval(temp['Raw Response'])[0]
driving_resp

{'bounds': {'northeast': {'lat': 40.9347924, 'lng': -73.808418},
  'southwest': {'lat': 40.7391335, 'lng': -74.00886710000002}},
 'copyrights': 'Map data ©2021 Google',
 'legs': [{'distance': {'text': '24.1 mi', 'value': 38833},
   'duration': {'text': '46 mins', 'value': 2752},
   'duration_in_traffic': {'text': '50 mins', 'value': 2986},
   'end_address': '809 Washington St, New York, NY 10014, USA',
   'end_location': {'lat': 40.7391335, 'lng': -74.0080765},
   'start_address': '120 Benchley Pl, Bronx, NY 10475, USA',
   'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
   'steps': [{'distance': {'text': '210 ft', 'value': 64},
     'duration': {'text': '1 min', 'value': 23},
     'end_location': {'lat': 40.87422850000001, 'lng': -73.8257921},
     'html_instructions': 'Head <b>northwest</b> toward <b>Benchley Pl</b>',
     'polyline': {'points': 'qhnxFlaraMA?A@A?A@A@A@?@A??@A@?@?@?@?@A??@?@?@?@@??@?@?@?@@??@?@@??@@??@@@@??@@?@@@?@?@?@?@?j@['},
     'start_loc

In [85]:
driving_resp.keys()

dict_keys(['bounds', 'copyrights', 'legs', 'overview_polyline', 'summary', 'warnings', 'waypoint_order'])

In [86]:
[print(k, type(v), str(v)[:150]) for k, v in driving_resp['legs'][0].items()]


distance <class 'dict'> {'text': '24.1 mi', 'value': 38833}
duration <class 'dict'> {'text': '46 mins', 'value': 2752}
duration_in_traffic <class 'dict'> {'text': '50 mins', 'value': 2986}
end_address <class 'str'> 809 Washington St, New York, NY 10014, USA
end_location <class 'dict'> {'lat': 40.7391335, 'lng': -74.0080765}
start_address <class 'str'> 120 Benchley Pl, Bronx, NY 10475, USA
start_location <class 'dict'> {'lat': 40.87448810000001, 'lng': -73.82567019999999}
steps <class 'list'> [{'distance': {'text': '210 ft', 'value': 64}, 'duration': {'text': '1 min', 'value': 23}, 'end_location': {'lat': 40.87422850000001, 'lng': -73.82579
traffic_speed_entry <class 'list'> []
via_waypoint <class 'list'> []


[None, None, None, None, None, None, None, None, None, None]

In [87]:
def summarize_leg_step(step):
    out = {}
    out['distance_mi'] = parse_distance(step['distance']['value'])
    out['duration_hr'] = parse_duration(step['duration']['value'])
    out['instructions'] = remove_html_tags(step['html_instructions'])
    
    out['distance_text'] = step['distance']['text']
    out['duration_text'] = step['duration']['text']
    
    out_series = pd.Series(out)
    return(out_series)

In [88]:
def consolidate_steps(driving_steps_dict):
    concise_steps = pd.DataFrame([summarize_leg_step(x) for x in driving_steps_dict])
    concise_steps = concise_steps.reset_index().rename({'index':'Step #'}, axis=1)
    concise_steps = concise_steps.sort_values('duration_hr', ascending=False)
    return(concise_steps)




In [89]:
legs = driving_resp['legs'][0]

In [90]:
concise_steps = consolidate_steps(legs['steps'])
concise_steps.iloc[:3]

,Step #,distance_mi,duration_hr,instructions,distance_text,duration_text
15,15,8.15,0.26,Keep right to stay on NY-9A S,8.2 mi,16 mins
14,14,4.66,0.10,Continue onto NY-9A S/Henry Hudson PkwyToll road,4.7 mi,6 mins
9,9,2.00,0.05,Take exit 6B to merge onto Cross County Pkwy t...,2.0 mi,3 mins


In [91]:
def parse_driving_details(driving_resp):
    
    legs = driving_resp['legs'][0]
    concise_steps = consolidate_steps(legs['steps'])

    driving_details = {}
    driving_details['summary'] = driving_resp['summary']
    driving_details['duration_hr'] = parse_duration(legs['duration']['value'])
    driving_details['duration_text'] = legs['duration']['text']

    driving_details['duration_in_traffic_hr'] = parse_duration(legs['duration_in_traffic']['value'])
    driving_details['duration_in_traffic_text'] = legs['duration_in_traffic']['text']
    
    driving_details['extra time in traffic_min'] = np.round((driving_details['duration_in_traffic_hr'] - driving_details['duration_hr'])*60.0, 2)
    
    driving_details['distance_mi'] = parse_distance(legs['distance']['value'])


    driving_details['1st Longest Step (time)'] = str(concise_steps.iloc[0].to_dict())
    driving_details['2nd Longest Step (time)'] = str(concise_steps.iloc[1].to_dict())
    driving_details['3rd Longest Step (time)'] = str(concise_steps.iloc[2].to_dict())

    return(pd.Series(driving_details))



In [92]:
driving_details1 = parse_driving_details(driving_resp)
driving_details1

summary                                                               NY-9A S
duration_hr                                                              0.76
duration_text                                                         46 mins
duration_in_traffic_hr                                                   0.83
duration_in_traffic_text                                              50 mins
time in traffic_min                                                       4.2
distance_mi                                                             24.13
1st Longest Step (time)     {'Step #': 15, 'distance_mi': 8.15, 'duration_...
2nd Longest Step (time)     {'Step #': 14, 'distance_mi': 4.66, 'duration_...
3rd Longest Step (time)     {'Step #': 9, 'distance_mi': 2.0, 'duration_hr...
dtype: object

## Sample Transit Parse

In [71]:
temp = df[(df['destination'] == rd.work) &
   (df['mode'] == 'transit')].iloc[1]
transit_resp = literal_eval(temp['Raw Response'])[0]
transit_resp

{'bounds': {'northeast': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
  'southwest': {'lat': 40.7378273, 'lng': -74.0080004}},
 'copyrights': 'Map data ©2021 Google',
 'legs': [{'arrival_time': {'text': '12:49pm',
    'time_zone': 'America/New_York',
    'value': 1630687792},
   'departure_time': {'text': '11:22am',
    'time_zone': 'America/New_York',
    'value': 1630682555},
   'distance': {'text': '16.0 mi', 'value': 25753},
   'duration': {'text': '1 hour 27 mins', 'value': 5237},
   'end_address': '809 Washington St, New York, NY 10014, USA',
   'end_location': {'lat': 40.7389303, 'lng': -74.0079928},
   'start_address': '120 Benchley Pl, Bronx, NY 10475, USA',
   'start_location': {'lat': 40.87448810000001, 'lng': -73.82567019999999},
   'steps': [{'distance': {'text': '0.3 mi', 'value': 537},
     'duration': {'text': '7 mins', 'value': 390},
     'end_location': {'lat': 40.8711287, 'lng': -73.82836329999999},
     'html_instructions': 'Walk to Asch Loop/Adler Pl',
  

In [72]:
def parse_transit_details(transit_resp):
    transit_details = {}
    legs = transit_resp['legs'][0]
    transit_steps = [x for x in legs['steps'] if x['travel_mode'] == 'TRANSIT']
    lines = [x['transit_details']['line']['short_name'] for x in transit_steps]
    transit_details['departure_time'] = parse_time(legs['departure_time']['value'])
    transit_details['arrival_time'] = parse_time(legs['arrival_time']['value'])
    transit_details['duration_hr'] = parse_duration(legs['duration']['value'])
    transit_details['duration_txt'] = legs['duration']['text']
    transit_details['distance_mi'] = parse_distance(legs['distance']['value'])
    transit_details['transit lines'] = lines
    transit_details['transit departure_time'] = [parse_time(x['transit_details']['departure_time']['value']) 
                                         for x in transit_steps]
    transit_details['transit arrival_time'] = [parse_time(x['transit_details']['arrival_time']['value']) 
                                         for x in transit_steps]
    transit_details['transit durations_hr'] = [parse_duration(x.get('duration').get('value')) 
                                            for x in transit_steps]
    transit_details = {k:str(v) if type(v) == list else v 
                      for k, v in transit_details.items()}
    return(pd.Series(transit_details))



In [73]:
transit_details1 = parse_transit_details(transit_resp)
transit_details1

departure_time                                          2021-09-03 15:22:35
arrival_time                                            2021-09-03 16:49:52
duration_hr                                                            1.45
duration_txt                                                 1 hour 27 mins
distance_mi                                                              16
transit lines                                            ['Bx28', '5', '2']
transit departure_time    ['2021-09-03 15:29:25', '2021-09-03 15:49:30',...
transit arrival_time      ['2021-09-03 15:38:37', '2021-09-03 16:15:00',...
transit durations_hr                                     [0.15, 0.42, 0.39]
dtype: object

## Demo Applying it 

In [74]:
df.columns

Index(['origin', 'destination', 'mode', 'departure_time', 'Raw Response',
       'Date', 'Time'],
      dtype='object')

In [75]:
df['mode'].value_counts()

driving    139
transit      9
Name: mode, dtype: int64

In [101]:
transits = df[df['mode'] == 'transit'] 
transit_df = transits['Raw Response']\
    .apply(lambda x: parse_transit_details(literal_eval(x)[0]))
transit_df



,departure_time,arrival_time,duration_hr,duration_txt,distance_mi,transit lines,transit departure_time,transit arrival_time,transit durations_hr
1,2021-09-03 13:34:23,2021-09-03 15:09:25,1.58,1 hour 35 mins,18.31,"['BxM7', 'E']","['2021-09-03 13:35:00', '2021-09-03 14:50:30']","['2021-09-03 14:38:00', '2021-09-03 15:00:30']","[1.05, 0.17]"
3,2021-09-03 15:22:35,2021-09-03 16:49:52,1.45,1 hour 27 mins,16.00,"['Bx28', '5', '2']","['2021-09-03 15:29:25', '2021-09-03 15:49:30',...","['2021-09-03 15:38:37', '2021-09-03 16:15:00',...","[0.15, 0.42, 0.39]"
5,2021-09-03 15:12:40,2021-09-03 16:46:48,1.57,1 hour 34 mins,16.49,"['Bx12-SBS', '6X', 'L']","['2021-09-03 15:16:08', '2021-09-03 15:40:00',...","['2021-09-03 15:23:00', '2021-09-03 16:30:00',...","[0.11, 0.83, 0.04]"
7,2021-09-03 15:15:34,2021-09-03 16:41:52,1.44,1 hour 26 mins,16.21,"['5', '2']","['2021-09-03 15:38:30', '2021-09-03 16:09:00']","['2021-09-03 16:07:00', '2021-09-03 16:32:30']","[0.48, 0.39]"
9,2021-09-03 15:23:19,2021-09-03 16:49:52,1.44,1 hour 27 mins,16.00,"['Bx28', '5', '2']","['2021-09-03 15:29:25', '2021-09-03 15:49:30',...","['2021-09-03 15:38:37', '2021-09-03 16:15:00',...","[0.15, 0.42, 0.39]"
11,2021-09-03 15:46:07,2021-09-03 17:11:48,1.43,1 hour 26 mins,16.49,"['Bx12-SBS', '6X', '5', 'L']","['2021-09-03 15:52:08', '2021-09-03 16:12:00',...","['2021-09-03 15:59:00', '2021-09-03 16:37:30',...","[0.11, 0.42, 0.26, 0.04]"
13,2021-09-03 16:01:59,2021-09-03 17:33:55,1.53,1 hour 32 mins,18.31,"['BxM7', 'E']","['2021-09-03 16:05:00', '2021-09-03 17:13:00']","['2021-09-03 17:03:00', '2021-09-03 17:25:00']","[0.97, 0.2]"
15,2021-09-03 16:01:59,2021-09-03 17:33:55,1.53,1 hour 32 mins,18.31,"['BxM7', 'E']","['2021-09-03 16:05:00', '2021-09-03 17:13:00']","['2021-09-03 17:03:00', '2021-09-03 17:25:00']","[0.97, 0.2]"
17,2021-09-03 16:01:59,2021-09-03 17:33:55,1.53,1 hour 32 mins,18.31,"['BxM7', 'E']","['2021-09-03 16:05:00', '2021-09-03 17:13:00']","['2021-09-03 17:03:00', '2021-09-03 17:25:00']","[0.97, 0.2]"


In [102]:
driving = df[df['mode'] == 'driving'] 
driving_df = driving['Raw Response']\
    .apply(lambda x: parse_driving_details(literal_eval(x)[0]))
driving_df

,summary,duration_hr,duration_text,duration_in_traffic_hr,duration_in_traffic_text,time in traffic_min,distance_mi,1st Longest Step (time),2nd Longest Step (time),3rd Longest Step (time)
0,I-95 S and NY-9A S,0.78,47 mins,0.77,46 mins,-0.6,18.57,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 4, 'distance_mi': 0.38, 'duration_h..."
2,NY-9A S,0.76,46 mins,0.83,50 mins,4.2,24.13,"{'Step #': 15, 'distance_mi': 8.15, 'duration_...","{'Step #': 14, 'distance_mi': 4.66, 'duration_...","{'Step #': 9, 'distance_mi': 2.0, 'duration_hr..."
4,NY-9A S,0.76,46 mins,0.84,50 mins,4.8,24.13,"{'Step #': 15, 'distance_mi': 8.15, 'duration_...","{'Step #': 14, 'distance_mi': 4.66, 'duration_...","{'Step #': 9, 'distance_mi': 2.0, 'duration_hr..."
6,NY-9A S,0.76,46 mins,0.84,51 mins,4.8,24.12,"{'Step #': 15, 'distance_mi': 8.15, 'duration_...","{'Step #': 14, 'distance_mi': 4.66, 'duration_...","{'Step #': 6, 'distance_mi': 0.78, 'duration_h..."
8,NY-9A S,0.76,46 mins,0.87,52 mins,6.6,24.13,"{'Step #': 15, 'distance_mi': 8.15, 'duration_...","{'Step #': 14, 'distance_mi': 4.66, 'duration_...","{'Step #': 9, 'distance_mi': 2.0, 'duration_hr..."
...,...,...,...,...,...,...,...,...,...,...
143,Hutchinson River Pkwy S,0.21,13 mins,0.21,13 mins,0.0,3.83,"{'Step #': 10, 'distance_mi': 0.36, 'duration_...","{'Step #': 12, 'distance_mi': 0.29, 'duration_...","{'Step #': 8, 'distance_mi': 0.94, 'duration_h..."
144,Orchard Beach Rd,0.22,13 mins,0.22,13 mins,0.0,3.83,"{'Step #': 5, 'distance_mi': 0.54, 'duration_h...","{'Step #': 3, 'distance_mi': 0.31, 'duration_h...","{'Step #': 13, 'distance_mi': 0.27, 'duration_..."
145,Hutchinson River Pkwy S,0.21,13 mins,0.20,12 mins,-0.6,3.83,"{'Step #': 10, 'distance_mi': 0.36, 'duration_...","{'Step #': 12, 'distance_mi': 0.29, 'duration_...","{'Step #': 8, 'distance_mi': 0.94, 'duration_h..."
146,Orchard Beach Rd,0.22,13 mins,0.22,13 mins,0.0,3.83,"{'Step #': 5, 'distance_mi': 0.54, 'duration_h...","{'Step #': 3, 'distance_mi': 0.31, 'duration_h...","{'Step #': 13, 'distance_mi': 0.27, 'duration_..."


In [104]:
driving_full = pd.merge(driving, driving_df
         , left_index=True, right_index=True)
transit_full = pd.merge(transits, transit_df
                       , left_index=True, right_index=True)
